# MODEL REGISTRY WITH MLFLOW

## SET UP MLFLOW EXPERIMENT TRACKING LOCAL HOST

In [1]:
import mlflow
from mlflow.tracking import MlflowClient

mlflow_tracking_uri = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=mlflow_tracking_uri)

### List all experiments and inspect some of them

In [2]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/6', experiment_id='6', lifecycle_stage='active', name='trip_prediction-experiment', tags={}>]

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = '6',
    filter_string = "metrics.rmse < 6.5",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 10,
    order_by = ['metric.rmse ASC']
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: c1db82cc4b45493a98c63bf0c8cc89e4, rmse: 6.3511
run id: 0200c2138adc4b76a41383d67c640a8c, rmse: 6.3511
run id: ed8ee5c154214b4980918ae620d9a6c5, rmse: 6.3693
run id: 82ca31957bdb48068a6662b1a02bb405, rmse: 6.3825
run id: f97291c0381d410da4ca2deb6af767b1, rmse: 6.4055
run id: 66877fe61b5f4b97876b7b433d1f8171, rmse: 6.4110
run id: 0c48c5b0a2e94828973dbacbaaf8e57f, rmse: 6.4444
run id: fb8c29338690490a900160af3e9f76c3, rmse: 6.4460
run id: 6d63f2b4d2b44784a23c0795f4ba02ec, rmse: 6.4728


### Interacting with the Model Registry

- set tracking uri
- define run id
- define model uri
- register model with register_model method
    - parameter
        1. model uri
        2. model name
        
This section use the **MlflowClient** instance to:

    1. Register a new version for the experiment "taxi_trip_regressor"
    2. Retrieve the latests versions of the model "taxi_trip_regressor" and check that a new version in this case "3" was created.
    3. Transition the version "3" to "Staging" and adding annotations to it.

**Registry model with MLflow ui**


`Registry base model`
!['base model'](https://github.com/surawut-jirasaktavee/course-mlops-zoomcamp/blob/main/local-host/02-experiment-tracking/images/Base_model.png)

`base model automate set to version 1`
!['version1'](https://github.com/surawut-jirasaktavee/course-mlops-zoomcamp/blob/main/local-host/02-experiment-tracking/images/version1.png)

`Registry model(Lasso) version 2 and switch to staging`
!['Lasso'](https://github.com/surawut-jirasaktavee/course-mlops-zoomcamp/blob/main/local-host/02-experiment-tracking/images/Registry_version2.png)

**Registry model with python**

In [7]:
mlflow.set_tracking_uri(mlflow_tracking_uri)

run_id = "c1db82cc4b45493a98c63bf0c8cc89e4"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="taxi_trip_regressor")

Registered model 'taxi_trip_regressor' already exists. Creating a new version of this model...
2022/06/18 23:18:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: taxi_trip_regressor, version 3
Created version '3' of model 'taxi_trip_regressor'.


<ModelVersion: creation_timestamp=1655569135884, current_stage='None', description=None, last_updated_timestamp=1655569135884, name='taxi_trip_regressor', run_id='c1db82cc4b45493a98c63bf0c8cc89e4', run_link=None, source='./mlruns/6/c1db82cc4b45493a98c63bf0c8cc89e4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

![Register model](https://github.com/surawut-jirasaktavee/course-mlops-zoomcamp/blob/main/local-host/02-experiment-tracking/images/Registry_version3.png)

In [8]:
model_name = "taxi_trip_regressor"
latest_versions = client.get_latest_versions(name=model_name)

model_versions = list()

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")
    model_versions.append(version.version)

version: 2, stage: Staging
version: 3, stage: None


In [10]:
model_versions

[2, 3]

In [11]:
selected_model = model_versions[-1]
model_stages = ["Staging", "Production", "Archived", "None"]
new_stage = model_stages[0]

client.transition_model_version_stage(
    name=model_name,
    version=selected_model,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1655569135884, current_stage='Staging', description=None, last_updated_timestamp=1655569930186, name='taxi_trip_regressor', run_id='c1db82cc4b45493a98c63bf0c8cc89e4', run_link=None, source='./mlruns/6/c1db82cc4b45493a98c63bf0c8cc89e4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

## Change model from Staging zone to Production

`Staging to Production with MLflow ui`
!['staging to prod'](https://github.com/surawut-jirasaktavee/course-mlops-zoomcamp/blob/main/local-host/02-experiment-tracking/images/staging-to-prod.png)


!['staging to prod'](https://github.com/surawut-jirasaktavee/course-mlops-zoomcamp/blob/main/local-host/02-experiment-tracking/images/staging-to-prod2.png)

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.

Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.



In [12]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)
        
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df.duration.apply(lambda x: x.total_seconds() / 60)
    
    df = df[(df['duration'] >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    
    return dv.transform(train_dicts)

def test_model(name, X_test, y_test, stage_name=None, version_no=None, model_name=None, stage=False, version=False, runs=False):
    
    '''
    Choose how to load model
    - version
    - stage
    - model flavor
    '''
    
    if version:
        model = mlflow.pyfunc.load_model(f"models:/{name}/{version}")
        y_pred = model.predict(X_test)
    elif runs:
        model = mlflow.pyfunc.load_model(f"runs:/{name}/{model_name}")
        y_pred = model.predict(X_test)
    elif stage:
        model = mlflow.pyfunc.load_model(f"models:/{name}/{stage_name}")
        y_pred = model.predict(X_test)
    
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}
    

In [13]:
mlflow.set_tracking_uri(mlflow_tracking_uri)

run_id = "c1db82cc4b45493a98c63bf0c8cc89e4"
model_name = "taxi_trip_regressor"

model_stages = ["Staging", "Production", "Archived", "None"]

In [14]:
test_path = './data/green_tripdata_2021-03.parquet'

In [15]:
df_test = read_dataframe(test_path)

In [16]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/premsurawut/_Github/MLOps/course-mlops-zoomcamp/local-host/02-experiment-tracking/preprocessor'

In [17]:
import pickle

with open('preprocessor/preprocessor.b', 'rb') as f_in:
    dv = pickle.load(f_in)

In [18]:
X_test = preprocess(df_test, dv)
X_test

<80372x13221 sparse matrix of type '<class 'numpy.float64'>'
	with 153356 stored elements in Compressed Sparse Row format>

In [19]:
target = 'duration'
y_test = df_test[target].values
y_test

array([ 8.35      ,  5.23333333, 20.33333333, ...,  9.        ,
       25.        ,  9.        ])

In [22]:
%time 

prod = test_model(name=model_name, X_test=X_test, y_test=y_test, stage_name="Production", stage=True) 

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 5.01 µs


In [24]:
%time 
new_model = test_model(name=run_id, X_test=X_test, y_test=y_test, model_name='models_xgboost', runs=True)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
[23:40:12] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [25]:
print("COMPARING BETWEEN THE CURRENT MODEL IN PRODUCTION AND NEW MODEL IN STAGING")
print(f"Prodcuction: Linear regression, {prod}")
print(f"Staging : XGBooster, {new_model}")

COMPARING BETWEEN THE CURRENT MODEL IN PRODUCTION AND NEW MODEL IN STAGING
Prodcuction: Linear regression, {'rmse': 7.720735231839729}
Staging : XGBooster, {'rmse': 6.295727269107297}


### PUT NEW MODEL TO THE PRODUCTION

In [27]:
def update_new_model(model_name, new_prod_ver, stage="Production", archive_existing_versions=False):
    client.transition_model_version_stage(
        name=model_name,
        version=new_prod_ver,
        stage=stage,
        archive_existing_versions=archive_existing_versions
    )
    return "Uploaded New model to Production successfully"

update_new_model(model_name, 3, archive_existing_versions=True)

'Uploaded New model to Production successfully'

### UPDATE MODEL STATUS

In [28]:
def update_model_metadata(model_name, new_prod_ver, old_prod_ver):
    
    from datetime import datetime
    date = datetime.today().date()
    
    
    client.update_model_version(
        name=model_name,
        version=new_prod_ver,
        description=f"Uploaded model to production on {date}")
    
    client.update_model_version(
        name=model_name,
        version=old_prod_ver,
        description=f"Transition model to archive on {date}")
    
    return f"Operation Succesfully"

In [29]:
update_model_metadata(model_name, 3, 1)

'Operation Succesfully'

**Switch New model to Production and Old Production to Archived**

!['new model to production](https://github.com/surawut-jirasaktavee/course-mlops-zoomcamp/blob/main/local-host/02-experiment-tracking/images/new_model-to-prod.png)